In [ ]:
# to connectto Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install umap-learn

In [ ]:
%matplotlib inline
import pandas as pd # for using pandas daraframe
import numpy as np # for som math operations
from numpy import save
from sklearn.decomposition import PCA # for PCA calculation
import matplotlib.pyplot as plt # for plotting
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import seaborn as sb
from collections import Counter
import time

# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

#UMAP
import umap

In [ ]:
#Funcion para graficar en 2D y 3D
def plot_2d(component1, component2,title):
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        text="{ "+df["City"]+" : " + df["Date"]+" }",
        mode='markers',
        marker=dict(
            size=5,
            color=labels, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=False,
            line_width=1,
        
        )
    ))
    fig.update_layout(xaxis_title='UMAP 1',
                    yaxis_title='UMAP 2',
                    title=title)                 
    fig.layout.template = 'seaborn'
    fig.show()

def plot_3d(component1,component2,component3,title):
    fig = go.Figure(data=[go.Scatter3d(
        x=component1,
        y=component2,
        z=component3,
        text="{ "+df["City"]+" : " + df["Date"]+" }",
        mode='markers',
        marker=dict(
            size=5,
            color=labels,                # set color to an array/list of desired values
            colorscale='Rainbow',   # choose a colorscale
            opacity=1,
            line_width=1
        )
    )])
    # tight layout
    fig.update_layout(scene = dict(xaxis_title='UMAP 1',
                    yaxis_title='UMAP 2',
                    zaxis_title='UMAP 3'),
                    title=title)
    fig.layout.template = 'seaborn'
    
    fig.show()

# Cargando datos

In [ ]:
## Cargando cvs
path = "/content/drive/MyDrive/CURSOS UNSA/Proyecto final de tesis/Avances Proyecto/Tesis_code/"

In [ ]:
## defining constants
PATH = path+"india_2015_2020.csv"
dataset = pd.read_csv(PATH)
df = dataset.copy()

In [ ]:
#Generando 
X_data = df.drop(columns=['City','AQI','AQI_Bucket','AQI','AQI_range'], inplace=False)
X_data.set_index('Date', inplace=True)

Y_labels = df.drop(columns=['Date','City','PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3','AQI','AQI_range'], inplace=False)

In [ ]:
# Dividiendo los datos en Datos y etiquetas
labels = Y_labels.loc[:,'AQI_Bucket'].values
data = X_data.loc[:,'PM2.5':].values

# Estandarización de datos

In [ ]:
## Standardizing the data
# StandardScaler(),RobustScaler(),MinMaxScaler()

# data_ = StandardScaler().fit_transform(data)
data_ = (X_data-X_data.min())/(X_data.max()-X_data.min())
data_

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3
Date,,,,,,,,,
2015-01-01,0.360461,0.662941,0.176961,0.153334,0.292380,0.163376,0.499343,0.060321,0.161687
2015-01-02,0.214185,0.293899,0.158863,0.138498,0.233027,0.153623,0.313403,0.043348,0.116250
2015-01-03,0.100196,0.143798,0.065788,0.127708,0.126771,0.335731,0.348555,0.017235,0.076440
2015-01-04,0.174646,0.263682,0.063945,0.155526,0.128543,0.629315,0.379106,0.030161,0.098363
2015-01-05,0.168613,0.238918,0.035787,0.147138,0.101126,0.593202,0.302234,0.021674,0.089981
...,...,...,...,...,...,...,...,...,...
2020-07-01,0.019965,0.033204,0.010035,0.055804,0.039208,0.035775,0.017740,0.043413,0.059716
2020-07-01,0.066218,0.110092,0.083911,0.063643,0.079843,0.085550,0.019382,0.022653,0.063907
2020-07-01,0.025147,0.037043,0.003763,0.040757,0.016920,0.040747,0.019382,0.070962,0.113068


# UMAP

In [ ]:
#Configration variable
dimensions = []
n_neighbors = [5, 10, 20, 50, 100, 200]
min_dist = [0.0, 0.1, 0.25, 0.5, 0.8, 0.99]
metric = ['euclidean', 'manhattan', 'canberra', 'mahalanobis', 'correlation','hamming']

In [ ]:
def draw_umap(n_neighbors=15, min_dist=0.1, n_components=3, metric='euclidean', title='',guardar="no"):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
    )
    start = time.time()
    u = fit.fit_transform(data);
    time_general.append(round(time.time() - start,2))
    if(guardar=="si"):
      #Guardar Componentes
      save(path+'umap_3.npy', u)

    #Plot 2D y 3D
    # plot_2d (u[:,0], u[:,1],title)
    # plot_3d (u[:, 0], u[:, 1],u[:, 2],title)

In [ ]:
# Mejor resultado
time_general =[]
draw_umap(n_neighbors=15,min_dist=0.25, n_components=3,metric='manhattan',title='Mejor UMAP',guardar="si")

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.



In [ ]:
time_general

[55.28]

In [ ]:
time_general =[]
for n in n_neighbors:
    draw_umap(n_neighbors=n, title='UMAP con n_neighbors = {}'.format(n))

In [ ]:
fig = go.Figure(data=go.Scatter(x=n_neighbors, y=time_general))
fig.update_layout(
    xaxis_title="n_neighbors",
    yaxis_title="Tiempo",
)
fig.show()

In [ ]:
time_general =[]
for d in min_dist:
    draw_umap(min_dist=d, title='min_dist = {}'.format(d))

In [ ]:
fig = go.Figure(data=go.Scatter(x=min_dist, y=time_general))
fig.update_layout(
    xaxis_title="min_dist",
    yaxis_title="Tiempo",
)
fig.show()

In [ ]:
time_general =[]
for m in metric:
  draw_umap(metric=m, title='metric = {}'.format(str(m)))

/usr/local/lib/python3.7/dist-packages/umap/umap_.py:1804: UserWarning:

gradient function is not yet implemented for hamming distance metric; inverse_transform will be unavailable



In [ ]:
fig = go.Figure(data=go.Scatter(x=metric, y=time_general))
fig.update_layout(
    xaxis_title="metric",
    yaxis_title="Tiempo",
)
fig.show()